# Imports

In [1]:
%matplotlib inline

import src.utils as udf

import gc
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import Lasso

import lightgbm as lgb
import xgboost as xgb

# Data

In [2]:
path = '../data/'

In [3]:
train = pd.read_pickle(f'{path}from_mod/train.pkl')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18264895 entries, 0 to 18264894
Data columns (total 18 columns):
building_id           uint16
meter                 uint8
meter_reading         float32
dew_temperature       float32
sea_level_pressure    float32
wind_speed            float32
primary_use           object
square_feet           uint32
year_built            uint16
missing_year          uint8
dayofyear             uint16
hour                  uint8
wind_direction_x      float32
wind_direction_y      float32
rel_humidity          float32
is_weekend            uint8
country               object
is_holiday            uint8
dtypes: float32(7), object(2), uint16(3), uint32(1), uint8(5)
memory usage: 1.1+ GB


In [4]:
train = udf.reduce_mem_usage(train)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18264895 entries, 0 to 18264894
Data columns (total 18 columns):
building_id           uint16
meter                 uint8
meter_reading         float32
dew_temperature       float32
sea_level_pressure    float32
wind_speed            float32
primary_use           object
square_feet           uint32
year_built            uint16
missing_year          uint8
dayofyear             uint16
hour                  uint8
wind_direction_x      float32
wind_direction_y      float32
rel_humidity          float32
is_weekend            uint8
country               object
is_holiday            uint8
dtypes: float32(7), object(2), uint16(3), uint32(1), uint8(5)
memory usage: 1.1+ GB


In [5]:
test = pd.read_pickle(f'{path}from_sub/test.pkl')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 17 columns):
building_id           uint16
meter                 uint8
dew_temperature       float32
sea_level_pressure    float32
wind_speed            float32
primary_use           object
square_feet           uint32
year_built            uint16
missing_year          uint8
dayofyear             int64
hour                  int64
wind_direction_x      float32
wind_direction_y      float32
rel_humidity          float32
is_weekend            int64
country               object
is_holiday            int64
dtypes: float32(6), int64(4), object(2), uint16(2), uint32(1), uint8(2)
memory usage: 3.5+ GB


In [6]:
test = udf.reduce_mem_usage(test)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 17 columns):
building_id           uint16
meter                 uint8
dew_temperature       float32
sea_level_pressure    float32
wind_speed            float32
primary_use           object
square_feet           uint32
year_built            uint16
missing_year          uint8
dayofyear             uint16
hour                  uint8
wind_direction_x      float32
wind_direction_y      float32
rel_humidity          float32
is_weekend            uint8
country               object
is_holiday            uint8
dtypes: float32(6), object(2), uint16(3), uint32(1), uint8(5)
memory usage: 2.4+ GB


In [7]:
submission = pd.read_csv(f'{path}raw/test/sample_submission.csv')
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
row_id           int64
meter_reading    int64
dtypes: int64(2)
memory usage: 636.3 MB


In [8]:
submission = udf.reduce_mem_usage(submission)
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
row_id           uint32
meter_reading    uint8
dtypes: uint32(1), uint8(1)
memory usage: 198.8 MB


In [9]:
mb = pd.read_pickle(f'{path}from_sub/mb.pkl')
mb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 9 columns):
row_id          uint32
building_id     uint16
meter           uint8
timestamp       datetime64[ns]
site_id         uint8
primary_use     object
square_feet     uint32
year_built      uint16
missing_year    uint8
dtypes: datetime64[ns](1), object(1), uint16(2), uint32(2), uint8(3)
memory usage: 1.5+ GB


In [10]:
mb = udf.reduce_mem_usage(mb)
mb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 9 columns):
row_id          uint32
building_id     uint16
meter           uint8
timestamp       datetime64[ns]
site_id         uint8
primary_use     object
square_feet     uint32
year_built      uint16
missing_year    uint8
dtypes: datetime64[ns](1), object(1), uint16(2), uint32(2), uint8(3)
memory usage: 1.5+ GB


In [11]:
del path
gc.collect()

44

# Features

In [12]:
train.head()

,building_id,meter,meter_reading,dew_temperature,sea_level_pressure,wind_speed,primary_use,square_feet,year_built,missing_year,dayofyear,hour,wind_direction_x,wind_direction_y,rel_humidity,is_weekend,country,is_holiday
0,46,0,15.604556,19.4,1019.400024,0.0,Retail,9045,2016,0,1,0,0.0,0.000000,100.000000,0,US,1
1,74,0,12.603682,19.4,1019.400024,0.0,Parking,387638,1997,0,1,0,0.0,0.000000,100.000000,0,US,1
2,93,0,15.364478,19.4,1019.400024,0.0,Office,33370,1982,0,1,0,0.0,0.000000,100.000000,0,US,1
3,105,0,23.303600,2.4,1020.900024,3.1,Education,50623,1960,1,1,0,-0.5,-0.866025,90.549408,0,UK,1
4,106,0,0.374600,2.4,1020.900024,3.1,Education,5374,1960,1,1,0,-0.5,-0.866025,90.549408,0,UK,1


In [13]:
test.head()

,building_id,meter,dew_temperature,sea_level_pressure,wind_speed,primary_use,square_feet,year_built,missing_year,dayofyear,hour,wind_direction_x,wind_direction_y,rel_humidity,is_weekend,country,is_holiday
0,0,0,12.8,1022.099976,2.1,Education,7432,2008,0,1,0,-0.642788,0.766044,83.409012,1,US,1
1,1,0,12.8,1022.099976,2.1,Education,2720,2004,0,1,0,-0.642788,0.766044,83.409012,1,US,1
2,2,0,12.8,1022.099976,2.1,Education,5376,1991,0,1,0,-0.642788,0.766044,83.409012,1,US,1
3,3,0,12.8,1022.099976,2.1,Education,23685,2002,0,1,0,-0.642788,0.766044,83.409012,1,US,1
4,4,0,12.8,1022.099976,2.1,Education,116607,1975,0,1,0,-0.642788,0.766044,83.409012,1,US,1


In [15]:
train.drop(['missing_year', 'wind_direction_x','sea_level_pressure', 'rel_humidity'], axis=1, inplace=True)
feats = train.drop('meter_reading', axis=1).columns
test = test[feats]
test.head()

,building_id,meter,dew_temperature,wind_speed,primary_use,square_feet,year_built,dayofyear,hour,wind_direction_y,is_weekend,country,is_holiday
0,0,0,12.8,2.1,Education,7432,2008,1,0,0.766044,1,US,1
1,1,0,12.8,2.1,Education,2720,2004,1,0,0.766044,1,US,1
2,2,0,12.8,2.1,Education,5376,1991,1,0,0.766044,1,US,1
3,3,0,12.8,2.1,Education,23685,2002,1,0,0.766044,1,US,1
4,4,0,12.8,2.1,Education,116607,1975,1,0,0.766044,1,US,1


In [16]:
del feats
gc.collect()

182

# Split data

In [17]:
train_df = []
test_df = []

for m in range(4):
    df_train = train[train.meter == m].drop('meter', axis=1)
    df_test = test[test.meter == m].drop('meter', axis=1)
    train_df.append(df_train)
    test_df.append(df_test)
    print(f'Meter {m}:', df_train.shape, df_test.shape)

Meter 0: (11530268, 13) (24755760, 12)
Meter 1: (3518870, 13) (8724960, 12)
Meter 2: (2296049, 13) (5676480, 12)
Meter 3: (919708, 13) (2540400, 12)


In [18]:
del m, df_train, df_test
gc.collect()

20

# Functions

In [19]:
def transform_data(df_train, df_test):
    X_train = df_train.drop('meter_reading', axis=1)
    y_train = df_train[['meter_reading']]
    y_train = np.log1p(y_train)
    
    X_train, X_val, X_test, rare_dict = udf.rare_encoder(X_train, df_test, ['primary_use'])
    X_train, X_val, X_test, mean_dict = udf.mean_encoder(X_train, y_train, X_test, ['primary_use', 'country'])
    X_train_scaled, X_val, X_test_scaled = udf.scale_feats(X_train, X_test)
    
    X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size=0.25, random_state=42)
    
    print('Train: ', X_train_scaled.shape, y_train.shape)
    print('Validation: ', X_val_scaled.shape, y_val.shape)
    print('Train: ', X_test_scaled.shape)
    
    return X_train_scaled, X_val_scaled, X_test_scaled, y_train, y_val

In [20]:
def predict_lgb(X_train, y_train, X_val, y_val, X_test, params_dict, save_path):
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)
    lgbm = lgb.train(params_dict, dtrain, valid_sets=[dtrain, dval], valid_names=['train', 'val'], verbose_eval=False)
    lgbm.save_model(save_path)
    
    pred = lgbm.predict(X_test)
    pred[pred < 0] = 0
    return pred

In [21]:
def predict_xgb(X_train, y_train, X_val, y_val, X_test, params_dict, save_path):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    xg = xgb.train(params_dict, dtrain, evals=[(dtrain, 'train'), (dval, 'val')], num_boost_round=1000, early_stopping_rounds=10, verbose_eval=False)
    xg.save_model(save_path)
    
    pred = xg.predict(dtest)
    pred[pred < 0] = 0
    return pred

In [22]:
gc.collect()

22

# Electricity meter

### Transform data

In [23]:
Xe_train_scaled, Xe_val_scaled, Xe_test_scaled, ye_train, ye_val = transform_data(train_df[0], test_df[0])
Xe_train_scaled.head()

/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Train:  (8647701, 12) (8647701, 1)
Validation:  (2882567, 12) (2882567, 1)
Train:  (24755760, 12)


,building_id,dew_temperature,wind_speed,primary_use,square_feet,year_built,dayofyear,hour,wind_direction_y,is_weekend,country,is_holiday
3820751,1.660735,-0.714804,0.663585,-0.364128,0.032242,-2.496025,-0.553622,0.794318,-0.425474,-0.633757,0.18362,-0.180316
5192151,-1.138221,0.185423,-0.197577,1.047964,0.306161,0.586150,-0.144126,-0.650406,1.588679,-0.633757,0.18362,-0.180316
10784538,-0.288186,0.519215,-0.886506,1.047964,-0.372759,-1.982329,1.484338,0.505373,-1.401239,-0.633757,0.18362,-0.180316
3949783,1.724795,-0.097795,-1.532377,-0.364128,-0.651102,-1.608732,-0.515530,1.661153,0.093720,-0.633757,0.18362,-0.180316
2900643,-1.071696,-0.209059,-0.197577,0.252910,-0.527437,-0.487941,-0.839318,0.505373,-0.941568,1.577891,0.18362,-0.180316


### Lasso Regression

In [24]:
lasso = Lasso(alpha=1e-6, random_state=42)
lasso.fit(Xe_train_scaled, ye_train)

val0 = lasso.predict(Xe_val_scaled)
print(lasso.score(Xe_val_scaled, ye_val))
print(np.sqrt(mean_squared_log_error(ye_val, val0)))

lasso0 = lasso.predict(Xe_test_scaled)
lasso0[lasso0 < 0] = 0
lasso0

0.43324185493903133
0.28364623098847


array([3.62139544, 3.5598328 , 3.48999142, ..., 3.37445649, 3.91047342,
       4.54247601])

### LightGBM

In [25]:
study_lgb0 = joblib.load('../objects/electricity/study_lgb.pkl')
params_lgb0 = study_lgb0.best_trial.params
params_lgb0

{'learning_rate': 0.07916336777546343,
 'lambda_l1': 0.008557356431137609,
 'lambda_l2': 0.0006037228650908533,
 'max_depth': 51,
 'num_leaves': 923,
 'min_child_samples': 7,
 'subsample': 0.7399597912518232,
 'feature_fraction': 0.7310599981838332}

In [26]:
params_lgb0['num_iterations'] = 10000
params_lgb0['early_stopping_round'] = 10
params_lgb0['metric'] = 'rmse'
params_lgb0['num_threads'] = -1
params_lgb0['seed'] = 42
params_lgb0

{'learning_rate': 0.07916336777546343,
 'lambda_l1': 0.008557356431137609,
 'lambda_l2': 0.0006037228650908533,
 'max_depth': 51,
 'num_leaves': 923,
 'min_child_samples': 7,
 'subsample': 0.7399597912518232,
 'feature_fraction': 0.7310599981838332,
 'num_iterations': 10000,
 'early_stopping_round': 10,
 'metric': 'rmse',
 'num_threads': -1,
 'seed': 42}

In [27]:
lgbm0 = predict_lgb(Xe_train_scaled, ye_train, Xe_val_scaled, ye_val, Xe_test_scaled, params_lgb0, '../objects/submission4/lgb0.pkl') # .txt')
lgbm0

/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


array([2.83301704, 1.71664249, 0.41793999, ..., 2.29911009, 5.20340237,
       1.66345259])

### XGBoost

In [28]:
study_xgb0 = joblib.load('../objects/electricity/study_xgb.pkl')
params_xgb0 = study_xgb0.best_trial.params
params_xgb0['eval_metric'] = 'rmse'
params_xgb0['seed'] = 42
params_xgb0

{'grow_policy': 'lossguide',
 'learning_rate': 0.014754037383886122,
 'alpha': 0.005261717289274988,
 'lambda': 0.10981652452119427,
 'gamma': 0.00015007710756172543,
 'max_depth': 23,
 'max_leaves': 1598,
 'subsample': 0.8406224713599797,
 'colsample_bytree': 0.9175035245877109,
 'eval_metric': 'rmse',
 'seed': 42}

In [29]:
xg0 = predict_xgb(Xe_train_scaled, ye_train, Xe_val_scaled, ye_val, Xe_test_scaled, params_xgb0, '../objects/submission4/xgb0.pkl') # .txt')
xg0

[02:13:58] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1572314959925/work/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


array([2.6438112, 1.7186092, 0.8908986, ..., 2.3611152, 5.1434426,
       1.6270531], dtype=float32)

In [30]:
del lasso, val0, study_lgb0
gc.collect()

100

# Chilled water meter

### Transform data

In [31]:
Xc_train_scaled, Xc_val_scaled, Xc_test_scaled, yc_train, yc_val = transform_data(train_df[1], test_df[1])
Xc_train_scaled.head()

/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Train:  (2639152, 12) (2639152, 1)
Validation:  (879718, 12) (879718, 1)
Train:  (8724960, 12)


,building_id,dew_temperature,wind_speed,primary_use,square_feet,year_built,dayofyear,hour,wind_direction_y,is_weekend,country,is_holiday
3054842,0.295110,-0.652177,-0.716347,0.758045,0.383649,-0.257998,1.245823,-0.088512,1.409013,-0.625362,-0.173847,-0.170259
1722336,0.817001,1.320539,0.043689,-1.578794,-0.856238,0.075476,-0.008720,-0.379102,-1.189395,-0.625362,-0.173847,-0.170259
511899,-1.397081,-0.760676,1.278746,0.611098,0.434406,0.408949,-1.213480,-1.250872,-1.526436,-0.625362,-0.173847,-0.170259
1159847,-1.891861,1.261357,0.281200,0.758045,-0.292965,1.687265,-0.526468,1.073849,1.508082,-0.625362,-0.173847,-0.170259
688211,0.277036,1.152858,0.518711,-1.578794,-0.386131,0.075476,-0.994432,-1.541463,0.761156,-0.625362,-0.173847,-0.170259


### Lasso Regression

In [32]:
lasso = Lasso(alpha=1e-6, random_state=42)
lasso.fit(Xc_train_scaled, yc_train)

val1 = lasso.predict(Xc_val_scaled)
print(lasso.score(Xc_val_scaled, yc_val))
print(np.sqrt(mean_squared_log_error(yc_val, val1)))

lasso1 = lasso.predict(Xc_test_scaled)
lasso1[lasso1 < 0] = 0
lasso1

0.29020340657972443
0.357727694438569


array([4.98783947, 4.05300896, 4.95738605, ..., 4.31064514, 4.15657049,
       4.47412741])

### LightGBM

In [33]:
study_lgb1 = joblib.load('../objects/chilledwater/study_lgb1.pkl')
params_lgb1 = study_lgb1.best_trial.params
params_lgb1

{'learning_rate': 0.020424864262841822,
 'lambda_l1': 0.27986455703148144,
 'lambda_l2': 7.08804479307531,
 'max_depth': 32,
 'num_leaves': 1293,
 'min_data_in_leaf': 23,
 'bagging_fraction': 0.9143657877950819,
 'feature_fraction': 0.701556769695671}

In [34]:
params_lgb1['num_iterations'] = 10000
params_lgb1['early_stopping_round'] = 20
params_lgb1['metric'] = 'rmse'
params_lgb1['num_threads'] = -1
params_lgb1['seed'] = 42
params_lgb1

{'learning_rate': 0.020424864262841822,
 'lambda_l1': 0.27986455703148144,
 'lambda_l2': 7.08804479307531,
 'max_depth': 32,
 'num_leaves': 1293,
 'min_data_in_leaf': 23,
 'bagging_fraction': 0.9143657877950819,
 'feature_fraction': 0.701556769695671,
 'num_iterations': 10000,
 'early_stopping_round': 20,
 'metric': 'rmse',
 'num_threads': -1,
 'seed': 42}

In [35]:
lgbm1 = predict_lgb(Xc_train_scaled, yc_train, Xc_val_scaled, yc_val, Xc_test_scaled, params_lgb1, '../objects/submission4/lgb1.pkl') # .txt')
lgbm1

/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


array([3.08155137, 0.23180519, 1.88365883, ..., 0.85868436, 0.37859159,
       1.65446393])

### XGBoost

In [36]:
study_xgb1 = joblib.load('../objects/chilledwater/study_xgb1.pkl')
params_xgb1 = study_xgb1.best_trial.params
params_xgb1['eval_metric'] = 'rmse'
params_xgb1['seed'] = 42
params_xgb1

{'grow_policy': 'depthwise',
 'learning_rate': 0.03801178677708901,
 'alpha': 0.051064916441743415,
 'lambda': 0.890189928673286,
 'gamma': 0.053083931710043705,
 'max_depth': 16,
 'max_leaves': 1131,
 'subsample': 0.7245353487626778,
 'colsample_bytree': 0.777530288110325,
 'eval_metric': 'rmse',
 'seed': 42}

In [37]:
xg1 = predict_xgb(Xc_train_scaled, yc_train, Xc_val_scaled, yc_val, Xc_test_scaled, params_xgb1, '../objects/submission4/xgb1.pkl') # .txt')
xg1

array([3.1646638 , 0.80339533, 2.354431  , ..., 0.75884616, 0.2715187 ,
       1.1792278 ], dtype=float32)

In [38]:
del lasso, val1, study_lgb1
gc.collect()

100

# Steam meter

### Transform data

In [39]:
Xs_train_scaled, Xs_val_scaled, Xs_test_scaled, ys_train, ys_val = transform_data(train_df[2], test_df[2])
Xs_train_scaled.head()

/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Train:  (1722036, 12) (1722036, 1)
Validation:  (574013, 12) (574013, 1)
Train:  (5676480, 12)


,building_id,dew_temperature,wind_speed,primary_use,square_feet,year_built,dayofyear,hour,wind_direction_y,is_weekend,country,is_holiday
1227863,-0.055769,1.149265,0.198853,0.693481,-0.370458,0.253185,0.140020,-1.520262,0.321329,-0.631491,-0.181765,-0.177586
793790,-1.072452,1.051160,-0.024691,-1.856454,-0.815237,0.118055,-0.531292,-0.072679,-0.784003,-0.631491,-0.181765,-0.177586
2268147,-0.794255,0.649821,-1.410663,-0.188679,-0.675412,-0.152204,1.641147,-1.375504,0.036405,-0.631491,-0.181765,-0.177586
970560,-0.065885,-0.188533,0.869485,1.900406,0.542175,-0.354898,-0.270226,1.664420,0.321329,-0.631491,-0.181765,-0.177586
1921836,-1.224196,1.051160,-1.410663,-1.856454,0.355498,0.118055,1.174958,-1.665020,0.036405,-0.631491,-0.181765,-0.177586


### Lasso Regression

In [40]:
lasso = Lasso(alpha=1e-6, random_state=42)
lasso.fit(Xs_train_scaled, ys_train)

val2 = lasso.predict(Xs_val_scaled)
print(lasso.score(Xs_val_scaled, ys_val))
print(np.sqrt(mean_squared_log_error(ys_val, val2)))

lasso2 = lasso.predict(Xs_test_scaled)
lasso2[lasso2 < 0] = 0
lasso2

0.3473431466224191
0.2689786952565921


array([5.30519067, 5.38990086, 5.42140252, ..., 5.96508901, 5.94411987,
       6.20910549])

### LightGBM

In [41]:
study_lgb2 = joblib.load('../objects/steam/study_lgb2.pkl')
params_lgb2 = study_lgb2.best_trial.params
params_lgb2

{'learning_rate': 0.04854268157540762,
 'lambda_l1': 0.0028266466239996284,
 'lambda_l2': 0.00010211303918218761,
 'max_depth': 56,
 'num_leaves': 409,
 'min_data_in_leaf': 8,
 'bagging_fraction': 0.5012612103606688,
 'feature_fraction': 0.8537936823847498}

In [42]:
params_lgb2['num_iterations'] = 10000
params_lgb2['early_stopping_round'] = 20
params_lgb2['metric'] = 'rmse'
params_lgb2['num_threads'] = -1
params_lgb2['seed'] = 42
params_lgb2

{'learning_rate': 0.04854268157540762,
 'lambda_l1': 0.0028266466239996284,
 'lambda_l2': 0.00010211303918218761,
 'max_depth': 56,
 'num_leaves': 409,
 'min_data_in_leaf': 8,
 'bagging_fraction': 0.5012612103606688,
 'feature_fraction': 0.8537936823847498,
 'num_iterations': 10000,
 'early_stopping_round': 20,
 'metric': 'rmse',
 'num_threads': -1,
 'seed': 42}

In [43]:
lgbm2 = predict_lgb(Xs_train_scaled, ys_train, Xs_val_scaled, ys_val, Xs_test_scaled, params_lgb2, '../objects/submission4/lgb2.pkl') # .txt')
lgbm2

/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


array([3.38614019, 5.26466672, 8.40859777, ..., 6.63733423, 6.46396314,
       3.54213874])

### XGBoost

In [44]:
study_xgb2 = joblib.load('../objects/steam/study_xgb2.pkl')
params_xgb2 = study_xgb2.best_trial.params
params_xgb2['eval_metric'] = 'rmse'
params_xgb2['seed'] = 42
params_xgb2

{'grow_policy': 'depthwise',
 'learning_rate': 0.045664576584020004,
 'alpha': 1.9071087639650297,
 'lambda': 0.00021496551872384197,
 'gamma': 0.15536767802205387,
 'max_depth': 59,
 'max_leaves': 1626,
 'subsample': 0.926581683911481,
 'colsample_bytree': 0.8620824662132568,
 'eval_metric': 'rmse',
 'seed': 42}

In [45]:
xg2 = predict_xgb(Xs_train_scaled, ys_train, Xs_val_scaled, ys_val, Xs_test_scaled, params_xgb2, '../objects/submission4/xgb2.pkl') # .txt')
xg2

array([3.4984896, 5.135393 , 8.485529 , ..., 6.504014 , 6.383502 ,
       3.4510038], dtype=float32)

In [46]:
del lasso, val2, study_lgb2
gc.collect()

100

# Hot water meter

### Transform data

In [47]:
Xh_train_scaled, Xh_val_scaled, Xh_test_scaled, yh_train, yh_val = transform_data(train_df[3], test_df[3])
Xh_train_scaled.head()

/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/utils/validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Train:  (689781, 12) (689781, 1)
Validation:  (229927, 12) (229927, 1)
Train:  (2540400, 12)


,building_id,dew_temperature,wind_speed,primary_use,square_feet,year_built,dayofyear,hour,wind_direction_y,is_weekend,country,is_holiday
617696,0.902166,1.246184,0.467919,0.356687,-0.296929,-0.002246,0.601279,0.219848,-1.311659,-0.630965,-0.044778,-0.182612
559728,-1.234496,1.653144,0.707877,-1.973171,-0.896851,2.317122,0.348563,0.654835,1.222356,-0.630965,-0.044778,-0.182612
900844,0.914025,-0.488753,0.227961,2.023683,-1.007877,-0.499254,1.648246,0.219848,0.372354,1.584873,-0.044778,5.476081
230300,-1.372855,0.089559,0.467919,-0.291586,-0.583896,-0.941038,-0.896966,0.799831,0.613943,-0.630965,-2.259887,-0.182612
268936,-1.129738,-0.842166,-0.011996,1.456112,-0.901935,0.605207,-0.770608,0.654835,-0.367554,-0.630965,-0.044778,-0.182612


### Lasso Regression

In [48]:
lasso = Lasso(alpha=1e-6, random_state=42)
lasso.fit(Xh_train_scaled, yh_train)

val3 = lasso.predict(Xh_val_scaled)
print(lasso.score(Xh_val_scaled, yh_val))
print(np.sqrt(mean_squared_log_error(yh_val, val3)))

lasso3 = lasso.predict(Xh_test_scaled)
lasso3[lasso3 < 0] = 0
lasso3

0.33942659336942604
0.39089294274292796


array([3.51657327, 3.87649386, 3.70830505, ..., 5.6030112 , 5.83196928,
       5.54659515])

### LightGBM

In [49]:
study_lgb3 = joblib.load('../objects/hotwater/study_lgb3.pkl')
params_lgb3 = study_lgb3.best_trial.params
params_lgb3

{'learning_rate': 0.023942434668217872,
 'lambda_l1': 0.23828824891615835,
 'lambda_l2': 0.00028375734074312625,
 'max_depth': 58,
 'num_leaves': 1581,
 'min_data_in_leaf': 42,
 'bagging_fraction': 0.500825513633077,
 'feature_fraction': 0.9607308095583501}

In [50]:
params_lgb3['num_iterations'] = 10000
params_lgb3['early_stopping_round'] = 20
params_lgb3['metric'] = 'rmse'
params_lgb3['num_threads'] = -1
params_lgb3['seed'] = 42

In [51]:
lgbm3 = predict_lgb(Xh_train_scaled, yh_train, Xh_val_scaled, yh_val, Xh_test_scaled, params_lgb3, '../objects/submission4/lgb3.pkl') # .txt')
lgbm3

/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/tri/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


array([2.27589531, 4.48015748, 4.68013297, ..., 8.35517495, 5.39646149,
       8.31210349])

### XGBoost

In [52]:
study_xgb3 = joblib.load('../objects/hotwater/study_xgb3.pkl')
params_xgb3 = study_xgb3.best_trial.params
params_xgb3['eval_metric'] = 'rmse'
params_xgb3['seed'] = 42
params_xgb3

{'grow_policy': 'lossguide',
 'learning_rate': 0.009806612868641755,
 'alpha': 0.009923410545051567,
 'lambda': 2.4314911765779557,
 'gamma': 0.00182351994095809,
 'max_depth': 51,
 'max_leaves': 1225,
 'subsample': 0.8446420358715253,
 'colsample_bytree': 0.8798084029338699,
 'eval_metric': 'rmse',
 'seed': 42}

In [53]:
xg3 = predict_xgb(Xh_train_scaled, yh_train, Xh_val_scaled, yh_val, Xh_test_scaled, params_xgb3, '../objects/submission4/xgb3.pkl') # .txt')
xg3

array([2.6411998, 4.477911 , 4.4737277, ..., 8.3466   , 5.6615877,
       8.247984 ], dtype=float32)

In [54]:
del lasso, val3, study_lgb3
gc.collect()

100

# Combine predictions

In [55]:
print('Test observations: ', [df.shape[0] for df in test_df])
print('Lasso predictions: ', list(map(len, [lasso0, lasso1, lasso2, lasso3])))
print('LightGBM predictions: ', list(map(len, [lgbm0, lgbm1, lgbm2, lgbm3])))
print('XGBoost predictions: ', list(map(len, [xg0, xg1, xg2, xg3])))

Test observations:  [24755760, 8724960, 5676480, 2540400]
Lasso predictions:  [24755760, 8724960, 5676480, 2540400]
LightGBM predictions:  [24755760, 8724960, 5676480, 2540400]
XGBoost predictions:  [24755760, 8724960, 5676480, 2540400]


In [56]:
# Electricity meter
pred0 = test_df[0][['building_id']].copy()
pred0['lasso'] = lasso0
pred0['lgb'] = lgbm0
pred0['xgb'] = xg0
pred0.drop('building_id', axis=1, inplace=True)
pred0

,lasso,lgb,xgb
0,3.621395,2.833017,2.643811
1,3.559833,1.716642,1.718609
2,3.489991,0.417940,0.890899
3,3.698655,3.209067,4.900446
4,4.188377,3.979461,3.308939
...,...,...,...
41697595,2.864940,1.633792,1.572824
41697596,3.878190,1.846834,1.828344
41697597,3.374456,2.299110,2.361115
41697598,3.910473,5.203402,5.143443


In [57]:
# Chilled water meter
pred1 = test_df[1][['building_id']].copy()
pred1['lasso'] = lasso1
pred1['lgb'] = lgbm1
pred1['xgb'] = xg1
pred1.drop('building_id', axis=1, inplace=True)
pred1

,lasso,lgb,xgb
8,4.987839,3.081551,3.164664
11,4.053009,0.231805,0.803395
16,4.957386,1.883659,2.354431
18,4.989375,1.811401,2.777094
20,4.075426,1.488554,2.215178
...,...,...,...
41697538,3.802885,3.592214,3.599005
41697541,4.016940,3.386084,3.408449
41697543,4.310645,0.858684,0.758846
41697545,4.156570,0.378592,0.271519


In [58]:
# Steam meter
pred2 = test_df[2][['building_id']].copy()
pred2['lasso'] = lasso2
pred2['lgb'] = lgbm2
pred2['xgb'] = xg2
pred2.drop('building_id', axis=1, inplace=True)
pred2

,lasso,lgb,xgb
16340505,5.305191,3.386140,3.498490
16340509,5.389901,5.264667,5.135393
16340515,5.421403,8.408598,8.485529
16340517,5.555816,4.991623,5.132029
16340521,5.443900,5.751615,5.763429
...,...,...,...
41697581,5.586897,6.060405,6.014267
41697584,5.805106,7.263215,7.015000
41697586,5.965089,6.637334,6.504014
41697588,5.944120,6.463963,6.383502


In [59]:
# Hot water meter
pred3 = test_df[3][['building_id']].copy()
pred3['lasso'] = lasso3
pred3['lgb'] = lgbm3
pred3['xgb'] = xg3
pred3.drop('building_id', axis=1, inplace=True)
pred3

,lasso,lgb,xgb
2260082,3.516573,2.275895,2.641200
2260086,3.876494,4.480157,4.477911
2260090,3.708305,4.680133,4.473728
2260092,4.173233,3.400146,3.154571
2260094,4.439970,5.577205,5.219765
...,...,...,...
41696866,5.658201,8.332293,8.331269
41697101,5.888385,5.428286,5.620398
41697116,5.603011,8.355175,8.346600
41697351,5.831969,5.396461,5.661588


In [60]:
# Log-transformed predictions
pred_transformed = pd.concat([pred0, pred1, pred2, pred3]).sort_index()
pred_transformed

,lasso,lgb,xgb
0,3.621395,2.833017,2.643811
1,3.559833,1.716642,1.718609
2,3.489991,0.417940,0.890899
3,3.698655,3.209067,4.900446
4,4.188377,3.979461,3.308939
...,...,...,...
41697595,2.864940,1.633792,1.572824
41697596,3.878190,1.846834,1.828344
41697597,3.374456,2.299110,2.361115
41697598,3.910473,5.203402,5.143443


In [61]:
del pred0, pred1, pred2, pred3
gc.collect()

110

# Transform predictions back to normal scale (and units for site 0)

In [62]:
# Transform back to normal scale
pred = pred_transformed.copy()
pred['lasso'] = np.expm1(pred.lasso)
pred['lgb'] = np.expm1(pred.lgb)
pred['xgb'] = np.expm1(pred.xgb)
pred.reset_index(inplace=True)
pred.columns = ['row_id', 'lasso', 'lgb', 'xgb']
pred

,row_id,lasso,lgb,xgb
0,0,36.389707,15.996663,13.066713
1,1,34.157318,4.565810,4.576767
2,2,31.785666,0.518830,1.437319
3,3,39.392939,23.755989,133.349747
4,4,64.915707,52.488182,26.356098
...,...,...,...,...
41697595,41697595,16.547994,4.123265,3.820243
41697596,41697596,47.336626,5.339714,5.223573
41697597,41697597,28.208404,8.965310,9.602770
41697598,41697598,48.922581,180.890048,170.304489


In [63]:
pred = pd.merge(mb[['row_id', 'site_id', 'meter']], pred, on='row_id', how='left')
pred

,row_id,site_id,meter,lasso,lgb,xgb
0,0,0,0,36.389707,15.996663,13.066713
1,1,0,0,34.157318,4.565810,4.576767
2,2,0,0,31.785666,0.518830,1.437319
3,3,0,0,39.392939,23.755989,133.349747
4,4,0,0,64.915707,52.488182,26.356098
...,...,...,...,...,...,...
41697595,41697595,15,0,16.547994,4.123265,3.820243
41697596,41697596,15,0,47.336626,5.339714,5.223573
41697597,41697597,15,0,28.208404,8.965310,9.602770
41697598,41697598,15,0,48.922581,180.890048,170.304489


In [64]:
# Site 0 meter 0 predictions in kWh
pred[(pred.site_id == 0) & (pred.meter == 0)].head()

,row_id,site_id,meter,lasso,lgb,xgb
0,0,0,0,36.389707,15.996663,13.066713
1,1,0,0,34.157318,4.565810,4.576767
2,2,0,0,31.785666,0.518830,1.437319
3,3,0,0,39.392939,23.755989,133.349747
4,4,0,0,64.915707,52.488182,26.356098


In [65]:
# Convert from kWh back to kBTU
pred = udf.convert_readings(pred, site_num=0, meter_type=0, conversion='kwh_to_kbtu', reading_col='lasso')
pred = udf.convert_readings(pred, site_num=0, meter_type=0, conversion='kwh_to_kbtu', reading_col='lgb')
pred = udf.convert_readings(pred, site_num=0, meter_type=0, conversion='kwh_to_kbtu', reading_col='xgb')
pred[(pred.site_id == 0) & (pred.meter == 0)].head()

,row_id,site_id,meter,lasso,lgb,xgb
0,0,0,0,124.154401,54.577415,44.581013
1,1,0,0,116.537939,15.577630,15.615013
2,2,0,0,108.446336,1.770143,4.903844
3,3,0,0,134.400830,81.050685,454.962646
4,4,0,0,221.479411,179.079178,89.921730


In [66]:
# Site 0 meter 1 predictions in tons
pred[(pred.site_id == 0) & (pred.meter == 1)].head()

,row_id,site_id,meter,lasso,lgb,xgb
8,8,0,1,145.619306,20.792184,22.680782
11,11,0,1,56.570424,0.260874,1.233110
16,16,0,1,141.221549,5.577527,9.532133
18,18,0,1,145.844645,5.119013,15.072246
20,20,0,1,57.875563,3.430683,8.163044


In [67]:
# Convert from kWh back to kBTU
pred = udf.convert_readings(pred, site_num=0, meter_type=1, conversion='ton_to_kbtu', reading_col='lasso')
pred = udf.convert_readings(pred, site_num=0, meter_type=1, conversion='ton_to_kbtu', reading_col='lgb')
pred = udf.convert_readings(pred, site_num=0, meter_type=1, conversion='ton_to_kbtu', reading_col='xgb')
pred[(pred.site_id == 0) & (pred.meter == 1)].head()

,row_id,site_id,meter,lasso,lgb,xgb
8,8,0,1,1747.431670,249.506208,272.169373
11,11,0,1,678.845088,3.130489,14.797322
16,16,0,1,1694.658589,66.930324,114.385597
18,18,0,1,1750.135745,61.428151,180.866943
20,20,0,1,694.506755,41.168198,97.956528


In [68]:
del mb
gc.collect()

110

# Save Predictions

In [69]:
pred.describe()

,row_id,site_id,meter,lasso,lgb,xgb
count,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07,4.169760e+07
mean,2.084880e+07,8.086134e+00,6.642857e-01,3.239290e+02,3.670009e+02,3.581980e+02
std,1.203706e+07,5.134712e+00,9.278067e-01,1.612918e+03,9.778432e+02,9.163020e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,2.221449e+00,0.000000e+00,0.000000e+00
25%,1.042440e+07,3.000000e+00,0.000000e+00,3.956849e+01,2.649503e+01,2.609304e+01
50%,2.084880e+07,9.000000e+00,0.000000e+00,7.585253e+01,8.794283e+01,8.717337e+01
75%,3.127320e+07,1.300000e+01,1.000000e+00,1.832989e+02,2.809187e+02,2.794785e+02
max,4.169760e+07,1.500000e+01,3.000000e+00,1.468159e+05,3.074340e+04,2.448289e+04


In [70]:
path = '../submissions/sub4/'
pred.to_pickle(f'{path}preds.pkl')
pred = pd.read_pickle(f'{path}preds.pkl')
pred

,row_id,site_id,meter,lasso,lgb,xgb
0,0,0,0,124.154401,54.577415,44.581013
1,1,0,0,116.537939,15.577630,15.615013
2,2,0,0,108.446336,1.770143,4.903844
3,3,0,0,134.400830,81.050685,454.962646
4,4,0,0,221.479411,179.079178,89.921730
...,...,...,...,...,...,...
41697595,41697595,15,0,16.547994,4.123265,3.820243
41697596,41697596,15,0,47.336626,5.339714,5.223573
41697597,41697597,15,0,28.208404,8.965310,9.602770
41697598,41697598,15,0,48.922581,180.890048,170.304489


In [71]:
submission

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
41697595,41697595,0
41697596,41697596,0
41697597,41697597,0
41697598,41697598,0


In [72]:
# Lasso predictions
lasso_pred = submission.copy()
lasso_pred['meter_reading'] = pred['lasso']
lasso_pred

,row_id,meter_reading
0,0,124.154401
1,1,116.537939
2,2,108.446336
3,3,134.400830
4,4,221.479411
...,...,...
41697595,41697595,16.547994
41697596,41697596,47.336626
41697597,41697597,28.208404
41697598,41697598,48.922581


In [73]:
# Lasso predictions
lgb_pred = submission.copy()
lgb_pred['meter_reading'] = pred['lgb']
lgb_pred

,row_id,meter_reading
0,0,54.577415
1,1,15.577630
2,2,1.770143
3,3,81.050685
4,4,179.079178
...,...,...
41697595,41697595,4.123265
41697596,41697596,5.339714
41697597,41697597,8.965310
41697598,41697598,180.890048


In [74]:
# Lasso predictions
xgb_pred = submission.copy()
xgb_pred['meter_reading'] = pred['xgb']
xgb_pred

,row_id,meter_reading
0,0,44.581013
1,1,15.615013
2,2,4.903844
3,3,454.962646
4,4,89.921730
...,...,...
41697595,41697595,3.820243
41697596,41697596,5.223573
41697597,41697597,9.602770
41697598,41697598,170.304489


In [75]:
lasso_pred.to_csv(f'{path}l1r.csv', index=False)
lasso_pred = pd.read_csv(f'{path}l1r.csv')
lasso_pred

,row_id,meter_reading
0,0,124.154401
1,1,116.537939
2,2,108.446336
3,3,134.400830
4,4,221.479411
...,...,...
41697595,41697595,16.547994
41697596,41697596,47.336626
41697597,41697597,28.208404
41697598,41697598,48.922581


In [76]:
lgb_pred.to_csv(f'{path}lgb.csv', index=False)
lgb_pred = pd.read_csv(f'{path}lgb.csv')
lgb_pred

,row_id,meter_reading
0,0,54.577415
1,1,15.577630
2,2,1.770143
3,3,81.050685
4,4,179.079178
...,...,...
41697595,41697595,4.123265
41697596,41697596,5.339714
41697597,41697597,8.965310
41697598,41697598,180.890048


In [77]:
xgb_pred.to_csv(f'{path}xgb.csv', index=False)
xgb_pred = pd.read_csv(f'{path}xgb.csv')
xgb_pred

,row_id,meter_reading
0,0,44.581013
1,1,15.615013
2,2,4.903844
3,3,454.962650
4,4,89.921730
...,...,...
41697595,41697595,3.820243
41697596,41697596,5.223573
41697597,41697597,9.602770
41697598,41697598,170.304490


### Test RMSLE:
##### Lasso regression:  (Public),  (Private)
##### LightGBM:  (Public),  (Private)
##### XGBoost:  (Public),  (Private)

# Combine LightGBM and XGBoost predictions

In [8]:
xl = pd.merge(lgb_pred, xgb_pred, on='row_id', how='left')
xl.columns = ['row_id', 'lgb', 'xgb']
xl.head()

,row_id,lgb,xgb
0,0,54.809699,51.882465
1,1,17.976733,16.754257
2,2,1.477563,3.712897
3,3,71.624491,373.013920
4,4,142.167250,117.079580


In [11]:
xl['xl55'] = xl[['lgb', 'xgb']].mean(axis=1)
xl['xl46'] = (xl.lgb * 0.4) + (xl.xgb * 0.6)
xl['xl2575'] = (xl.lgb * 0.25) + (xl.xgb * 0.75)
xl.head()

,row_id,lgb,xgb,xl55
0,0,54.809699,51.882465,53.346082
1,1,17.976733,16.754257,17.365495
2,2,1.477563,3.712897,2.595230
3,3,71.624491,373.013920,222.319205
4,4,142.167250,117.079580,129.623415


In [16]:
xl1 = xl[['row_id', 'xl55']]
xl1.columns = ['row_id', 'meter_reading']

xl1.to_csv(f'{path}xl1.csv', index=False)
xl1 = pd.read_csv(f'{path}xl1.csv')
xl1

,row_id,meter_reading
0,0,53.346082
1,1,17.365495
2,2,2.595230
3,3,222.319205
4,4,129.623415
...,...,...
41697595,41697595,4.897745
41697596,41697596,5.232897
41697597,41697597,7.730367
41697598,41697598,172.912913


In [17]:
xl2 = xl[['row_id', 'xl46']]
xl2.columns = ['row_id', 'meter_reading']

xl2.to_csv(f'{path}xl2.csv', index=False)
xl2 = pd.read_csv(f'{path}xl2.csv')
xl2

,row_id,meter_reading
0,0,53.053358
1,1,17.243247
2,2,2.818763
3,3,252.458148
4,4,127.114648
...,...,...
41697595,41697595,4.863569
41697596,41697596,5.207736
41697597,41697597,7.822719
41697598,41697598,172.888550


In [18]:
xl3 = xl[['row_id', 'xl2575']]
xl3.columns = ['row_id', 'meter_reading']

xl3.to_csv(f'{path}xl3.csv', index=False)
xl3 = pd.read_csv(f'{path}xl3.csv')
xl3

,row_id,meter_reading
0,0,52.614273
1,1,17.059876
2,2,3.154063
3,3,297.666563
4,4,123.351498
...,...,...
41697595,41697595,4.812304
41697596,41697596,5.169994
41697597,41697597,7.961248
41697598,41697598,172.852006
